In [3]:
import json

# JSON 파일 경로
file_path = "../app/data/book_embeddings_openai.json"

# JSON 파일 불러오기
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)  # JSON을 파이썬 딕셔너리/리스트로 변환


In [5]:
book_idx = data.keys()

In [6]:
import numpy as np
embedding_list = []
isbn_list = []
for idx in book_idx:
  embedding_list.append(data[idx]['embedding'])
  isbn_list.append(data[idx]['isbn'])
embedding_arr = np.array(embedding_list)
isbn_arr = np.array(isbn_list)

In [8]:
np.save('embedding_embedding.npy', embedding_arr)
np.save('cosine_similarity.npy', embedding_arr)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(embedding_arr)


In [235]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def filter_top_70_percent(embedding_arr, original_indices):
    """
    현재 벡터들에서 코사인 유사도를 기반으로 상위 70% 벡터만 남기는 함수
    """
    cos_sim = cosine_similarity(embedding_arr)
    
    # ✅ 랜덤으로 벡터 하나 선택 (현재 남아있는 벡터 중)
    random_idx = np.random.choice(len(embedding_arr), 1)[0]

    # ✅ 선택된 벡터와 유사한 순서대로 정렬
    sorted_indices = np.argsort(-cos_sim[random_idx])

    # ✅ 상위 70% 벡터만 선택
    num_keep = int(len(sorted_indices) * 0.57)
    top_indices = sorted_indices[:num_keep]

    # ✅ 벡터와 인덱스 업데이트
    return embedding_arr[top_indices], original_indices[top_indices]

# ✅ 초기 데이터 설정
embedding_arr = np.array(embedding_list)
original_indices = np.arange(len(embedding_arr))

# ✅ 10번 반복하여 상위 70% 유지
for round_num in range(10):
    print(f"\n🔹 {round_num+1}번째 라운드: 현재 벡터 개수 = {len(embedding_arr)}")
    
    embedding_arr, original_indices = filter_top_70_percent(embedding_arr, original_indices)

print(f"\n✅ 최종 남은 벡터 개수: {len(embedding_arr)}")
print(f"✅ 최종 선택된 원본 벡터 인덱스: {original_indices}")



🔹 1번째 라운드: 현재 벡터 개수 = 1781

🔹 2번째 라운드: 현재 벡터 개수 = 1015

🔹 3번째 라운드: 현재 벡터 개수 = 578

🔹 4번째 라운드: 현재 벡터 개수 = 329

🔹 5번째 라운드: 현재 벡터 개수 = 187

🔹 6번째 라운드: 현재 벡터 개수 = 106

🔹 7번째 라운드: 현재 벡터 개수 = 60

🔹 8번째 라운드: 현재 벡터 개수 = 34

🔹 9번째 라운드: 현재 벡터 개수 = 19

🔹 10번째 라운드: 현재 벡터 개수 = 10

✅ 최종 남은 벡터 개수: 5
✅ 최종 선택된 원본 벡터 인덱스: [ 538 1076  256 1120   31]
